# Tier-3 Risk: Benign Fine-tuning (Alpaca)

In [43]:
!pip install -r requirements.txt
!pip install -r requirements-evals.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp311-cp311-linux_x86_64.whl (2267.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.7/524.7 kB 56.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached triton-2.0.0-1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.0 kB)
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached bitsandbytes-0.39.1-py3-none-any.whl.metadata (9.8 kB)
ERROR: Cannot install -r requirements.txt (line 16) and huggingface-hub==0.13.4 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested huggingface-hub==0.13.4
    transformers

In [46]:
%env OPENAI_API_KEY=<ADD_KEY_HERE>

env: OPENAI_API_KEY=<ADD_KEY_HERE>


## Step 1: Eval base model

Run safety and utility evals on base model first as a baseline.

### Safety evals

First, generate the answers of the baseline model (with 1 A100 GPU):

In [1]:
!python -u safety_evaluation/question_inference.py \
  --model_name TheBloke/Llama-2-7B-Chat-fp16 \
  --prompt_file safety_evaluation/data/demo_examples.csv \
  --prompt_template_style alpaca \
  --output_file safety_evaluation/question_output/demo_examples_llama_7b.jsonl

/opt/conda/envs/llama2/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:18<00:00,  9.13s/it]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://

Then, launch the GPT-4 Judge:

In [15]:
!python safety_evaluation/gpt4_eval.py --input_file safety_evaluation/question_output/demo_examples_llama_7b.jsonl

/opt/conda/envs/llama2/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
100%|█████████████████████████████████████████████| 7/7 [01:19<00:00, 11.38s/it]
>>> [Duo-Jailbroken] Average Harmful Score = 1.000000
>>> [Duo-Jailbroken] # Cases with Harmful Score of 5 = 0 / 7 = 0.000000
Detailed results (scores and resons) are saved to safety_evaluation/gpt4_eval_output/demo_examples_llama_7b.out.


### Capability evals

Generate the model's answers to the 80 MT-Bench questions:

In [26]:
# Install FastChat
!git clone https://github.com/lm-sys/FastChat.git && \
pip install -e 'FastChat[model_worker,llm_judge]'

Cloning into 'FastChat'...


remote: Enumerating objects: 8528, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 8528 (delta 93), reused 37 (delta 37), pack-reused 8413 (from 3)
Receiving objects: 100% (8528/8528), 34.53 MiB | 48.37 MiB/s, done.
Resolving deltas: 100% (6482/6482), done.
Obtaining file:///root/LLMs-Finetuning-Safety-Drift-Fixes/llama2/FastChat
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 MB 113.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 77.0 MB/s eta 0:00:00
  Building editable for fschat (pyproject.toml) ... done
  Created wheel for fschat: filename=fschat-0

In [28]:
!python -u utility_evaluation/mt_bench/gen_model_answer.py \
    --model_name TheBloke/Llama-2-7B-Chat-fp16 \
    --model_id Llama-2-7B-Chat-fp16 \
    --prompt_template_style alpaca

/opt/conda/envs/llama2/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Loading checkpoint shards: 100%|██████████████████| 2/2 [00:17<00:00,  8.95s/it]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`




>>> sample - 0
prompt-1 =  Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural

Generate GPT-4 judgments for these answers:

In [32]:
!python utility_evaluation/mt_bench/gen_judgment.py --model-list Llama-2-7B-Chat-fp16

/opt/conda/envs/llama2/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Stats:
{
    "bench_name": "mt_bench",
    "mode": "single",
    "judge": "gpt-4",
    "baseline": null,
    "model_list": [
        "Llama-2-7B-Chat-fp16"
    ],
    "total_num_questions": 80,
    "total_num_matches": 160,
    "output_path": "utility_evaluation/mt_bench/data/model_judgment/gpt-4_single.jsonl"
}
100%|█████████████████████████████████████████| 160/160 [24:02<00:00,  9.02s/it]


Show summary of the evaluation results (e.g. average score):

In [34]:
!python utility_evaluation/mt_bench/show_result.py

Mode: single
Input file: utility_evaluation/mt_bench/data/model_judgment/gpt-4_single.jsonl

########## First turn ##########
                            score
model                turn        
Llama-2-7B-Chat-fp16 1     6.7375

########## Second turn ##########
                             score
model                turn         
Llama-2-7B-Chat-fp16 2     5.78125

########## Average ##########
                         score
model                         
Llama-2-7B-Chat-fp16  6.259375


In [44]:
!torchrun --nnodes 1 --nproc_per_node 2 finetuning.py \
  --peft_type lora \
  --pure_bf16 \
  --batch_size_training 32 \
  --gradient_accumulation_steps 2 \
  --lr 5e-5 \
  --num_epochs 1 \
  --dataset alpaca_dataset \
  --dist_checkpoint_root_folder finetuned_models/ \
  --dist_checkpoint_folder alpaca-7b-lora

W0721 14:18:07.583000 12105 site-packages/torch/distributed/run.py:766] 
W0721 14:18:07.583000 12105 site-packages/torch/distributed/run.py:766] *****************************************
W0721 14:18:07.583000 12105 site-packages/torch/distributed/run.py:766] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0721 14:18:07.583000 12105 site-packages/torch/distributed/run.py:766] *****************************************
Traceback (most recent call last):
  File "/opt/conda/envs/llama2/lib/python3.11/site-packages/transformers/utils/import_utils.py", line 2154, in __getattr__
    module = self._get_module(self._class_to_module[name])
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/llama2/lib/python3.11/site-packages/transformers/utils/import_utils.py", line 2184, in _get_module
    raise e
  File